# The index building process

In [2]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE  SCHEDULED_STOP
,cluster-0016  GCE       2                                             RUNNING  us-central1-a


# Imports & Setup

In [3]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

,WARNING: Running pip as the 'root' user can result in broken permissions and conflicting behaviour with the system package manager, possibly rendering your system unusable.It is recommended to use a virtual environment instead: https://pip.pypa.io/warnings/venv. Use the --root-user-action option if you know what you are doing and want to suppress this warning.
,

In [4]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
,[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan  8 15:04 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [6]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [7]:
spark

In [8]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'roi-ir-bucket-1919' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh':
        paths.append(full_path+b.name)

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [9]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("text", "id").rdd
doc2title = parquetFile.select("id", "title").rdd

In [10]:
# saving RDD in the form of dict to retrive titles to their doc id
doc2title_dict = doc2title.collectAsMap()
with open('/./home/dataproc/doc2title_dict.pkl', 'wb') as file:
    pickle.dump(doc2title_dict, file)

!gsutil cp /home/dataproc/doc2title_dict.pkl gs://roi-ir-bucket-1919/

Copying file:///home/dataproc/doc2title_dict.pkl [Content-Type=application/octet-stream]...
,==> NOTE: You are uploading one or more large file(s), which would run          
,significantly faster if you enable parallel composite uploads. This
,feature can be enabled by editing the
,"parallel_composite_upload_threshold" value in your .boto
,configuration file. However, note that if you do this large files will
,be uploaded as `composite objects
,<https://cloud.google.com/storage/docs/composite-objects>`_,which
,means that any user who downloads such objects will need to have a
,compiled crcmod installed (see "gsutil help crcmod"). This is because
,without a compiled crcmod, computing checksums on composite objects is
,so slow that gsutil disables downloads of composite objects.
,
,| [1 files][168.9 MiB/168.9 MiB]                                                
,Operation completed over 1 objects/168.9 MiB.                                    


In [11]:
print(list(doc2title_dict.items())[:2])

[(4045403, 'Foster Air Force Base'), (4045413, 'Torino Palavela')]


We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [12]:
# Count number of wiki pages
parquetFile.count()

6348910

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [13]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [14]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [15]:
from inverted_index_gcp import InvertedIndex

**Tokenization & More**

In [16]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in
  `all_stopwords` and return entries that will go into our posting lists.
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  filtered_tokens = []
  word_count_list = []
  for token in tokens:
    if token not in all_stopwords: #first we take out all the stop words
      filtered_tokens.append(token)
  word_count = {} #we create a dict for the word counts
  for token in filtered_tokens:
    word_count[token] = word_count.get(token, 0) + 1 #we count each word 
  for token, count in word_count.items(): #we loop over the dict
    word_count_list.append((token, (id, count))) #then append to list
  return word_count_list #according to the asked pattern

def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  sorted_posting_list = sorted(unsorted_pl, key=lambda x: x[0])
  return sorted_posting_list

def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  return postings.mapValues(lambda posting_list: len(posting_list)) #we count the number of times each word apear in a document


def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out
  all posting lists in a bucket to disk, and returns the posting locations for
  each bucket. Partitioning should be done through the use of `token2bucket`
  above. Writing to disk should use the function  `write_a_posting_list`, a
  static method implemented in inverted_index_colab.py under the InvertedIndex
  class.
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and
      offsets its posting list was written to. See `write_a_posting_list` for
      more details.
  '''
  # first we map each token to the bucket (bucket id, (token,posting_list))
  token_to_bucket = postings.map(lambda x: (token2bucket_id(x[0]),(x[0],x[1])))
  #we group each token by bucket id 
  group_by_bucketID = token_to_bucket.groupByKey()
  # we write each bucket to the disk
  # we pack (bucket_id, list_of_terms) into one tuple 
  # and then write_a_posting_list wirtes the bucket to file and returns
  # a dictionary mapping each token to the locations of its posting list
  bucket_to_disk = group_by_bucketID.map(lambda bucket: 
                                         InvertedIndex.write_a_posting_list(
                                             (bucket[0],list(bucket[1])),bucket_name))
  return bucket_to_disk

In [ ]:
# time the index creation time
t_start = time()
# word counts map
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
doc_len_rdd = doc_text_pairs.map(
    lambda x: (x[0], len(str(x[1]).split()))
)
doc_lengths = doc_text_pairs.map(lambda x: len(str(x[1]).split()))
avg_doc_length = doc_lengths.mean()
with open("avg_doc_len.pkl", "wb") as f:
    pickle.dump(avg_doc_length, f)

!gsutil cp avg_doc_len.pkl gs://roi-ir-bucket-1919/avg_doc_len.pkl

# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered).collect()
index_const_time = time() - t_start

Copying file://avg_doc_len.pkl [Content-Type=application/octet-stream]...
,/ [1 files][   21.0 B/   21.0 B]                                                
,Operation completed over 1 objects/21.0 B.                                       


In [18]:
# test index construction time
assert index_const_time < 60*120

In [19]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

Putting it all together

In [20]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'index')
# upload to gs
index_src = "index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://index.pkl [Content-Type=application/octet-stream]...
,/ [1 files][ 18.4 MiB/ 18.4 MiB]                                                
,Operation completed over 1 objects/18.4 MiB.                                     


In [21]:
!gsutil ls -lh $index_dst

 18.45 MiB  2026-01-09T20:32:41Z  gs://roi-ir-bucket-1919/postings_gcp/index.pkl
,TOTAL: 1 objects, 19343760 bytes (18.45 MiB)


# PageRank

**YOUR TASK (10 POINTS):** Compute PageRank for the entire English Wikipedia. Use your implementation for `generate_graph` function from Colab below.

In [ ]:
def generate_graph(pages):
  ''' Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and
      'anchor_text'.
  Returns:
  --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the
      second entry is the destination page id. No duplicates should be present.
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph
      created by the wikipedia links. No duplicates should be present.
  '''
  # we create the edeges and for each page (row), we produce pairs where each
  # is holding source_id and destination_id
  edges = pages.flatMap(lambda row: [(row.id, link.id) for link in row.anchor_text])
  # we remove duplicates
  edges = edges.distinct()
  # we create a set of vertices that appear as source pages
  source_vertices = pages.map(lambda row:(row.id,))
  # we create a set of vertices that appear as destination pages
  # can have multiple links
  destination_vertices = pages.flatMap(lambda row: [(link.id,) for link in row.anchor_text])
  # we combine them and remove duplicates
  vertices = source_vertices.union(destination_vertices).distinct()
  return edges, vertices


In [ ]:
t_start = time()
pages_links = spark.read.parquet("gs://roi-ir-bucket-1919/multistream*").select("id", "anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr_rdd = pr_results.vertices \
    .select("id", "pagerank") \
    .rdd \
    .map(lambda r: (int(r.id), float(r.pagerank)))

# Convert to dictionary
pagerank_dict = pr_rdd.collectAsMap()

# Save locally as a pickle file
with open("pagerank.pkl", "wb") as f:
    pickle.dump(pagerank_dict, f)

# Upload to GCP bucket using gsutil
dest_path = f"gs://{bucket_name}/pagerank.pkl"
!gsutil cp pagerank.pkl $dest_path

pr_time = time() - t_start
pr_results.vertices.select("id", "pagerank").show(5)

print(f"PageRank Finished! Time: {pr_time:.2f}s")
print(f"Total pages ranked: {len(pagerank_dict)}")

In [ ]:
t_start = time()
pages_links = spark.read.parquet("gs://roi-ir-bucket-1919/multistream*").select("id", "anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr_rdd = pr_results.vertices \
    .select("id", "pagerank") \
    .rdd \
    .map(lambda r: (int(r.id), float(r.pagerank)))

# Convert to dictionary
pagerank_dict = pr_rdd.collectAsMap()

# Save locally as a pickle file
with open("pagerank.pkl", "wb") as f:
    pickle.dump(pagerank_dict, f)

# Upload to GCP bucket using gsutil
dest_path = f"gs://{bucket_name}/pagerank.pkl"
!gsutil cp pagerank.pkl $dest_path

pr_time = time() - t_start
pr_results.vertices.select("id", "pagerank").show(5)

print(f"PageRank Finished! Time: {pr_time:.2f}s")
print(f"Total pages ranked: {len(pagerank_dict)}")

Copying file://pagerank.pkl [Content-Type=application/octet-stream]...
,| [1 files][ 84.7 MiB/ 84.7 MiB]                                                
,Operation completed over 1 objects/84.7 MiB.                                     


+----+------------------+
,|  id|          pagerank|
,+----+------------------+
,|1677| 40.60299390449796|
,|1697| 6.725054306196098|
,|1806| 71.23858462582163|
,|1950| 94.42358111798255|
,|2250|2.5674204862854544|
,+----+------------------+
,only showing top 5 rows
,
,PageRank Finished! Time: 6328.52s
,Total pages ranked: 6348983


In [ ]:
# test that PageRank computaion took less than 1 hour
assert pr_time < 60*120